# 새로운 노래
* 전 주에 없던 새로운 노래 확인

## 라이브러리

In [1]:
import pandas as pd
import os

## 코드 현재 경로

In [2]:
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

# Flo

In [3]:
# 파일 불러오기
folder_path = '/crawled_data/live_flo/'
path = code_path + folder_path
try:
    os.chdir(path)
except:
    pass

In [4]:
# 파일 이름들
file_list = os.listdir()
file_list

['live_flo_20220405.xlsx',
 'live_flo_20220406.xlsx',
 'live_flo_20220407.xlsx',
 'live_flo_20220412.xlsx',
 'live_flo_20220413.xlsx',
 'live_flo_20220414_110035.xlsx',
 'live_flo_20220415_110026.xlsx',
 'live_flo_20220418_110024.xlsx',
 'live_flo_20220419_110028.xlsx',
 'live_flo_20220420_110013.xlsx',
 'live_flo_20220421_110051.xlsx',
 'live_flo_20220422_110026.xlsx',
 'live_flo_20220423_110023.xlsx',
 'live_flo_20220425_110022.xlsx',
 'live_flo_20220426_110029.xlsx',
 'live_flo_20220427_110027.xlsx',
 'live_flo_20220428_110009.xlsx',
 'live_flo_20220429_110008.xlsx']

In [80]:
# 전 주 파일 날짜
start_date = '2022-04-18'
end_date = '2022-04-23'
mod_start_date = start_date.replace('-', '')
mod_end_date = end_date.replace('-', '')

## 새로운 날짜 설정

In [81]:
# 비교할 파일 날짜
new_date = '2022-04-25'
mod_new_date = new_date.replace('-', '')

In [82]:
for i, one_file in enumerate(file_list):
    if mod_start_date == mod_end_date:
        start_idx = i
        end_idx = i
    else:
        if mod_start_date in one_file:
            start_idx = i
        elif mod_end_date in one_file:
            end_idx = i
mod_file_list = file_list[start_idx:end_idx+1]
mod_file_list

['live_vibe_20220418_111004.xlsx',
 'live_vibe_20220419_110642.xlsx',
 'live_vibe_20220420_110627.xlsx',
 'live_vibe_20220421_110704.xlsx',
 'live_vibe_20220422_110647.xlsx']

In [83]:
df = pd.DataFrame()
for one in mod_file_list:
    temp = pd.read_excel(one)
    df = pd.concat([df, temp], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      500 non-null    object
 1   순위      500 non-null    int64 
 2   곡       500 non-null    object
 3   가수      500 non-null    object
 4   앨범      500 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.7+ KB


In [84]:
title_list = df['곡'].unique()
mod_title_list = [x.strip() for x in title_list]
mod_title_list[:5]

['봄여름가을겨울 (Still Life)',
 'TOMBOY',
 'LOVE DIVE',
 'Feel My Rhythm',
 'GANADARA (Feat. 아이유)']

In [85]:
# 새로운 곡 목록
for i, one_file in enumerate(file_list):
    if mod_new_date in one_file:
        new_idx = i
new_file_name = file_list[new_idx]
new_file_name

'live_vibe_20220425_110704.xlsx'

In [86]:
new_df = pd.read_excel(new_file_name)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      100 non-null    object
 1   순위      100 non-null    int64 
 2   곡       100 non-null    object
 3   가수      100 non-null    object
 4   앨범      100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [87]:
new_title_list = new_df['곡'].unique()
mod_new_title_list = [x.strip() for x in new_title_list]

In [88]:
new_df['곡'] = mod_new_title_list

In [89]:
# 새로운 곡
new_pop_title_list = []
for one in mod_new_title_list:
    if one not in mod_title_list:
        new_pop_title_list.append(one)
len(new_pop_title_list)

1

In [90]:
new_pop_title_list

['정이라고 하자 (Feat. 10CM)']

In [91]:
new_pop_df = pd.DataFrame()
for one in new_pop_title_list:
    temp = new_df[new_df['곡'] == one]
    new_pop_df = pd.concat([new_pop_df, temp], ignore_index=True)
new_pop_df

,날짜,순위,곡,가수,앨범
0,2022-04-25,41,정이라고 하자 (Feat. 10CM),BIG Naughty (서동현),정이라고 하자


# bugs

In [17]:
# 파일 불러오기
folder_path = '/crawled_data/live_bugs/'
path = code_path + folder_path
try:
    os.chdir(path)
except:
    pass

In [18]:
# 파일 이름들
file_list = os.listdir()
file_list

['live_bugs_20220422_105549.xlsx',
 'live_bugs_20220425_110806.xlsx',
 'live_bugs_20220426_110837.xlsx',
 'live_bugs_20220427_110836.xlsx',
 'live_bugs_20220428_110800.xlsx',
 'live_bugs_20220429_110800.xlsx']

In [23]:
# 전 주 파일 날짜
start_date = '2022-04-22'
end_date = '2022-04-28'
mod_start_date = start_date.replace('-', '')
mod_end_date = end_date.replace('-', '')

## 새로운 날짜 설정

In [24]:
# 비교할 파일 날짜
new_date = '2022-04-29'
mod_new_date = new_date.replace('-', '')

In [25]:
for i, one_file in enumerate(file_list):
    if mod_start_date == mod_end_date:
        start_idx = i
        end_idx = i
    else:
        if mod_start_date in one_file:
            start_idx = i
        elif mod_end_date in one_file:
            end_idx = i
mod_file_list = file_list[start_idx:end_idx+1]
mod_file_list

['live_bugs_20220422_105549.xlsx',
 'live_bugs_20220425_110806.xlsx',
 'live_bugs_20220426_110837.xlsx',
 'live_bugs_20220427_110836.xlsx',
 'live_bugs_20220428_110800.xlsx']

In [26]:
df = pd.DataFrame()
for one in mod_file_list:
    temp = pd.read_excel(one)
    df = pd.concat([df, temp], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      500 non-null    object
 1   순위      500 non-null    int64 
 2   곡       500 non-null    object
 3   가수      500 non-null    object
 4   앨범      500 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.7+ KB


In [27]:
title_list = df['곡'].unique()
mod_title_list = [x.strip() for x in title_list]
mod_title_list[:5]

['LOVE DIVE', 'Seoul', '그대가 있기에', '봄여름가을겨울 (Still Life)', 'Love story']

In [28]:
# 새로운 곡 목록
for i, one_file in enumerate(file_list):
    if mod_new_date in one_file:
        new_idx = i
new_file_name = file_list[new_idx]
new_file_name

'live_bugs_20220429_110800.xlsx'

In [29]:
new_df = pd.read_excel(new_file_name)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      100 non-null    object
 1   순위      100 non-null    int64 
 2   곡       100 non-null    object
 3   가수      100 non-null    object
 4   앨범      100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [30]:
new_title_list = new_df['곡'].unique()
mod_new_title_list = [x.strip() for x in new_title_list]

In [31]:
new_df['곡'] = mod_new_title_list

In [32]:
# 새로운 곡
new_pop_title_list = []
for one in mod_new_title_list:
    if one not in mod_title_list:
        new_pop_title_list.append(one)
len(new_pop_title_list)

32

In [33]:
new_pop_title_list

['C.I.T.T (Cheese in the Trap)',
 'I Am',
 '비 오는 날 듣기 좋은 노래 (Feat. Colde)',
 '선물',
 'INVU (ZHU Remix)',
 'INDUSTRY BABY',
 'How You Like That',
 '마음 (Feat. 적재)',
 '우리의 이야기',
 '그때 그 아인',
 'Sisyphus',
 'What If',
 'Loadstar',
 '동화',
 '문어의 꿈',
 '천년의 사랑',
 'Lovers On The Moon',
 '내 맘대로 (#Self_Trip)',
 '알았다면',
 '나를 외치다',
 '인사',
 '저녁 하늘',
 'BOP BOP! (Yves V Remix)',
 '오르막길',
 '반달 (My Moon)',
 '부끄럼',
 '좋은 날',
 '그대가 살아서',
 '짙어져',
 '하루가 달리',
 'IF I',
 '찾았다']

In [35]:
new_pop_df = pd.DataFrame()
for one in new_pop_title_list:
    temp = new_df[new_df['곡'] == one]
    new_pop_df = pd.concat([new_pop_df, temp], ignore_index=True)
new_pop_df

,날짜,순위,곡,가수,앨범
0,2022-04-29,26,C.I.T.T (Cheese in the Trap),문별 (마마무),C.I.T.T (Cheese in the Trap)
1,2022-04-29,38,I Am,Nicholas Cheung,I Am
2,2022-04-29,48,비 오는 날 듣기 좋은 노래 (Feat. Colde),에픽하이 (EPIK HIGH),비 오는 날 듣기 좋은 노래
3,2022-04-29,54,선물,멜로망스(MeloMance),Moonlight
4,2022-04-29,57,INVU (ZHU Remix),태연 (TAEYEON),iScreaM Vol.15 : INVU Remixes
5,2022-04-29,60,INDUSTRY BABY,Lil Nas X,INDUSTRY BABY
6,2022-04-29,61,How You Like That,BLACKPINK,How You Like That
7,2022-04-29,65,마음 (Feat. 적재),소유 (SOYOU),The 1st Mini Album 'Day & Night'
8,2022-04-29,66,우리의 이야기,멜로망스(MeloMance),유미의 세포들 OST Part 8
9,2022-04-29,68,그때 그 아인,김필(Kim Feel),이태원 클라쓰 OST Part.6


# genie

In [36]:
# 파일 불러오기
folder_path = '/crawled_data/live_genie/'
path = code_path + folder_path
try:
    os.chdir(path)
except:
    pass

In [37]:
# 파일 이름들
file_list = os.listdir()
file_list

['live_genie_20220405.xlsx',
 'live_genie_20220406.xlsx',
 'live_genie_20220407.xlsx',
 'live_genie_20220408.xlsx',
 'live_genie_20220412.xlsx',
 'live_genie_20220413.xlsx',
 'live_genie_20220413_152916.xlsx',
 'live_genie_20220414_134752.xlsx',
 'live_genie_20220415_180602.xlsx',
 'live_genie_20220418_110238.xlsx',
 'live_genie_20220419_110232.xlsx',
 'live_genie_20220420_110217.xlsx',
 'live_genie_20220421_110254.xlsx',
 'live_genie_20220422_110235.xlsx',
 'live_genie_20220425_110255.xlsx',
 'live_genie_20220426_110231.xlsx',
 'live_genie_20220427_110229.xlsx',
 'live_genie_20220428_110201.xlsx',
 'live_genie_20220429_110201.xlsx']

In [38]:
# 전 주 파일 날짜
start_date = '2022-04-18'
end_date = '2022-04-22'
mod_start_date = start_date.replace('-', '')
mod_end_date = end_date.replace('-', '')

## 새로운 날짜 설정

In [39]:
# 비교할 파일 날짜
new_date = '2022-04-25'
mod_new_date = new_date.replace('-', '')

In [40]:
for i, one_file in enumerate(file_list):
    if mod_start_date == mod_end_date:
        start_idx = i
        end_idx = i
    else:
        if mod_start_date in one_file:
            start_idx = i
        elif mod_end_date in one_file:
            end_idx = i
mod_file_list = file_list[start_idx:end_idx+1]
mod_file_list

['live_genie_20220418_110238.xlsx',
 'live_genie_20220419_110232.xlsx',
 'live_genie_20220420_110217.xlsx',
 'live_genie_20220421_110254.xlsx',
 'live_genie_20220422_110235.xlsx']

In [41]:
df = pd.DataFrame()
for one in mod_file_list:
    temp = pd.read_excel(one)
    df = pd.concat([df, temp], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      500 non-null    object
 1   순위      500 non-null    int64 
 2   곡       500 non-null    object
 3   가수      500 non-null    object
 4   앨범      500 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.7+ KB


In [42]:
title_list = df['곡'].unique()
mod_title_list = [x.strip() for x in title_list]
mod_title_list[:5]

['봄여름가을겨울 (Still Life)', 'LOVE DIVE', 'TOMBOY', 'Feel My Rhythm', '사랑인가 봐']

In [43]:
# 새로운 곡 목록
for i, one_file in enumerate(file_list):
    if mod_new_date in one_file:
        new_idx = i
new_file_name = file_list[new_idx]
new_file_name

'live_genie_20220425_110255.xlsx'

In [44]:
new_df = pd.read_excel(new_file_name)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      100 non-null    object
 1   순위      100 non-null    int64 
 2   곡       100 non-null    object
 3   가수      100 non-null    object
 4   앨범      100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [45]:
new_title_list = new_df['곡'].unique()
mod_new_title_list = [x.strip() for x in new_title_list]

In [46]:
new_df['곡'] = mod_new_title_list

In [47]:
# 새로운 곡
new_pop_title_list = []
for one in mod_new_title_list:
    if one not in mod_title_list:
        new_pop_title_list.append(one)
len(new_pop_title_list)

9

In [48]:
new_pop_title_list

['나의 X에게',
 '살기 위해서',
 '자가격리 (Feat. Paloalto)',
 '봄 to 러브',
 '선물',
 '정이라고 하자 (Feat. 10CM)',
 '에잇 (Prod. & Feat. SUGA of BTS)',
 'Every Second',
 "That's Hilarious"]

In [49]:
new_pop_df = pd.DataFrame()
for one in new_pop_title_list:
    temp = new_df[new_df['곡'] == one]
    new_pop_df = pd.concat([new_pop_df, temp], ignore_index=True)
new_pop_df

,날짜,순위,곡,가수,앨범
0,2022-04-25,14,나의 X에게,경서,나의 X에게
1,2022-04-25,31,살기 위해서,순순희,살기 위해서
2,2022-04-25,37,자가격리 (Feat. Paloalto),릴러말즈 (Leellamarz) & TOIL,자가격리
3,2022-04-25,72,봄 to 러브,10CM,우리들의 블루스 OST Part 3
4,2022-04-25,77,선물,멜로망스 (MeloMance),Moonlight
5,2022-04-25,78,정이라고 하자 (Feat. 10CM),BIG Naughty (서동현),정이라고 하자
6,2022-04-25,91,에잇 (Prod. & Feat. SUGA of BTS),아이유 (IU),에잇
7,2022-04-25,93,Every Second,Mina Okabe,Every Second
8,2022-04-25,96,That's Hilarious,Charlie Puth,That's Hilarious


# melon

In [52]:
# 파일 불러오기
folder_path = '/crawled_data/live_melon/'
path = code_path + folder_path
try:
    os.chdir(path)
except:
    pass

In [53]:
# 파일 이름들
file_list = os.listdir()
file_list

['live_melon_20220405.xlsx',
 'live_melon_20220406.xlsx',
 'live_melon_20220407.xlsx',
 'live_melon_20220412.xlsx',
 'live_melon_20220413.xlsx',
 'live_melon_20220413_153438.xlsx',
 'live_melon_20220414_134807.xlsx',
 'live_melon_20220415_180604.xlsx',
 'live_melon_20220418_110427.xlsx',
 'live_melon_20220419_110436.xlsx',
 'live_melon_20220420_110419.xlsx',
 'live_melon_20220421_110456.xlsx',
 'live_melon_20220422_110438.xlsx',
 'live_melon_20220425_110456.xlsx',
 'live_melon_20220426_110432.xlsx',
 'live_melon_20220427_110430.xlsx',
 'live_melon_20220428_110400.xlsx',
 'live_melon_20220429_110400.xlsx']

In [54]:
# 전 주 파일 날짜
start_date = '2022-04-18'
end_date = '2022-04-22'
mod_start_date = start_date.replace('-', '')
mod_end_date = end_date.replace('-', '')

## 새로운 날짜 설정

In [55]:
# 비교할 파일 날짜
new_date = '2022-04-25'
mod_new_date = new_date.replace('-', '')

In [56]:
for i, one_file in enumerate(file_list):
    if mod_start_date == mod_end_date:
        start_idx = i
        end_idx = i
    else:
        if mod_start_date in one_file:
            start_idx = i
        elif mod_end_date in one_file:
            end_idx = i
mod_file_list = file_list[start_idx:end_idx+1]
mod_file_list

['live_melon_20220418_110427.xlsx',
 'live_melon_20220419_110436.xlsx',
 'live_melon_20220420_110419.xlsx',
 'live_melon_20220421_110456.xlsx',
 'live_melon_20220422_110438.xlsx']

In [57]:
df = pd.DataFrame()
for one in mod_file_list:
    temp = pd.read_excel(one)
    df = pd.concat([df, temp], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      500 non-null    object
 1   순위      500 non-null    int64 
 2   곡       500 non-null    object
 3   가수      500 non-null    object
 4   앨범      500 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.7+ KB


In [58]:
title_list = df['곡'].unique()
mod_title_list = [x.strip() for x in title_list]
mod_title_list[:5]

['봄여름가을겨울 (Still Life)', 'TOMBOY', 'LOVE DIVE', 'Feel My Rhythm', '사랑인가 봐']

In [59]:
# 새로운 곡 목록
for i, one_file in enumerate(file_list):
    if mod_new_date in one_file:
        new_idx = i
new_file_name = file_list[new_idx]
new_file_name

'live_melon_20220425_110456.xlsx'

In [60]:
new_df = pd.read_excel(new_file_name)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      100 non-null    object
 1   순위      100 non-null    int64 
 2   곡       100 non-null    object
 3   가수      100 non-null    object
 4   앨범      100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [61]:
new_title_list = new_df['곡'].unique()
mod_new_title_list = [x.strip() for x in new_title_list]

In [62]:
new_df['곡'] = mod_new_title_list

In [63]:
# 새로운 곡
new_pop_title_list = []
for one in mod_new_title_list:
    if one not in mod_title_list:
        new_pop_title_list.append(one)
len(new_pop_title_list)

4

In [64]:
new_pop_title_list

['나의 X에게', 'With You', '자가격리 (Feat. Paloalto)', '봄 to 러브']

In [65]:
new_pop_df = pd.DataFrame()
for one in new_pop_title_list:
    temp = new_df[new_df['곡'] == one]
    new_pop_df = pd.concat([new_pop_df, temp], ignore_index=True)
new_pop_df

,날짜,순위,곡,가수,앨범
0,2022-04-25,40,나의 X에게,경서,나의 X에게
1,2022-04-25,42,With You,지민,우리들의 블루스 OST Part 4
2,2022-04-25,77,자가격리 (Feat. Paloalto),릴러말즈 (Leellamarz),자가격리
3,2022-04-25,93,봄 to 러브,10CM,우리들의 블루스 OST Part 3


# vibe

In [66]:
# 파일 불러오기
folder_path = '/crawled_data/live_vibe/'
path = code_path + folder_path
try:
    os.chdir(path)
except:
    pass

In [67]:
# 파일 이름들
file_list = os.listdir()
file_list

['live_vibe_20220405.xlsx',
 'live_vibe_20220406.xlsx',
 'live_vibe_20220407.xlsx',
 'live_vibe_20220412.xlsx',
 'live_vibe_20220413.xlsx',
 'live_vibe_20220413_154819.xlsx',
 'live_vibe_20220414_134945.xlsx',
 'live_vibe_20220415_180611.xlsx',
 'live_vibe_20220418_111004.xlsx',
 'live_vibe_20220419_110642.xlsx',
 'live_vibe_20220420_110627.xlsx',
 'live_vibe_20220421_110704.xlsx',
 'live_vibe_20220422_110647.xlsx',
 'live_vibe_20220425_110704.xlsx',
 'live_vibe_20220426_110637.xlsx',
 'live_vibe_20220427_110635.xlsx',
 'live_vibe_20220428_110605.xlsx',
 'live_vibe_20220429_110625.xlsx']

In [68]:
# 전 주 파일 날짜
start_date = '2022-04-18'
end_date = '2022-04-22'
mod_start_date = start_date.replace('-', '')
mod_end_date = end_date.replace('-', '')

## 새로운 날짜 설정

In [69]:
# 비교할 파일 날짜
new_date = '2022-04-25'
mod_new_date = new_date.replace('-', '')

In [70]:
for i, one_file in enumerate(file_list):
    if mod_start_date == mod_end_date:
        start_idx = i
        end_idx = i
    else:
        if mod_start_date in one_file:
            start_idx = i
        elif mod_end_date in one_file:
            end_idx = i
mod_file_list = file_list[start_idx:end_idx+1]
mod_file_list

['live_vibe_20220418_111004.xlsx',
 'live_vibe_20220419_110642.xlsx',
 'live_vibe_20220420_110627.xlsx',
 'live_vibe_20220421_110704.xlsx',
 'live_vibe_20220422_110647.xlsx']

In [71]:
df = pd.DataFrame()
for one in mod_file_list:
    temp = pd.read_excel(one)
    df = pd.concat([df, temp], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      500 non-null    object
 1   순위      500 non-null    int64 
 2   곡       500 non-null    object
 3   가수      500 non-null    object
 4   앨범      500 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.7+ KB


In [72]:
title_list = df['곡'].unique()
mod_title_list = [x.strip() for x in title_list]
mod_title_list[:5]

['봄여름가을겨울 (Still Life)',
 'TOMBOY',
 'LOVE DIVE',
 'Feel My Rhythm',
 'GANADARA (Feat. 아이유)']

In [73]:
# 새로운 곡 목록
for i, one_file in enumerate(file_list):
    if mod_new_date in one_file:
        new_idx = i
new_file_name = file_list[new_idx]
new_file_name

'live_vibe_20220425_110704.xlsx'

In [74]:
new_df = pd.read_excel(new_file_name)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   날짜      100 non-null    object
 1   순위      100 non-null    int64 
 2   곡       100 non-null    object
 3   가수      100 non-null    object
 4   앨범      100 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [75]:
new_title_list = new_df['곡'].unique()
mod_new_title_list = [x.strip() for x in new_title_list]

In [76]:
new_df['곡'] = mod_new_title_list

In [77]:
# 새로운 곡
new_pop_title_list = []
for one in mod_new_title_list:
    if one not in mod_title_list:
        new_pop_title_list.append(one)
len(new_pop_title_list)

1

In [78]:
new_pop_title_list

['정이라고 하자 (Feat. 10CM)']

In [79]:
new_pop_df = pd.DataFrame()
for one in new_pop_title_list:
    temp = new_df[new_df['곡'] == one]
    new_pop_df = pd.concat([new_pop_df, temp], ignore_index=True)
new_pop_df

,날짜,순위,곡,가수,앨범
0,2022-04-25,41,정이라고 하자 (Feat. 10CM),BIG Naughty (서동현),정이라고 하자


# 코드 마지막